In [1]:
import logging
import os
import json
import time

import h5py
import numpy as np

from annoy import AnnoyIndex

import torch
import torch.nn as nn
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
from PIL import Image

In [2]:
print(os.getcwd()) # C:\Users\chung\Documents\04-Insight\insight\notebooks
os.chdir('../')

print(os.getcwd()) # C:\Users\chung\Documents\04-Insight\insight

C:\Users\chung\Documents\04-Insight\insight\notebooks
C:\Users\chung\Documents\04-Insight\insight


In [3]:
# specify architecture to be used
arch = 'resnet18'
model_file = 'places365/%s_places365.pth.tar' %arch

#load pre-trained weights
model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k, 'module.',''): v for k, v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.fc_backup = model.fc
model.fc = nn.Sequential()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# define image transformer
center_crop = trn.Compose([trn.Resize((256,256)),
                               trn.CenterCrop(224),
                               trn.ToTensor(),
                               trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                              ])

In [5]:
# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
# load the class label
file_name = 'places365/categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

In [7]:
img_name = 'notebooks/data/arch/49927265438.jpg'
img = Image.open(img_name)
input_img = V(center_crop(img).unsqueeze(0))
logit = model.forward(input_img)

In [10]:
logit.shape

torch.Size([1, 512])